# Audio Compiler

This notebook compiles individual audio samples into combined audio files for tape recording.

**Purpose:**
- Load audio samples from a source directory
- Normalize to various amplitude levels (data augmentation)
- Add spoken digit markers for batch identification
- Add audio delimiters (beeps, silence) for alignment
- Combine into batches with metadata (JSON)

**Output:**
- `combined_files_XXX.wav` - Combined audio batches
- `combined_files_XXX.json` - Metadata with segment boundaries

## 1. Imports and Dependencies

In [1]:
import json
import os
import subprocess
import sys

# Add project root to path for imports
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))

import librosa
from pydub import AudioSegment

from src.utils import Config

## 2. Utility Functions

In [2]:
class Util:
    @staticmethod
    def find_files_by_extension(dir_path, extension, recursive=True):
        files = []
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if os.path.isfile(file_path) and file_path.endswith(extension):
                files.append(file_path)
            elif os.path.isdir(file_path) and recursive:
                files.extend(Util.find_files_by_extension(file_path, extension, recursive))
        return files

    @staticmethod
    def find_folders_by_extension(dir_path, recursive=False):
        folders = []
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if os.path.isdir(file_path):
                folders.append(file_path)
                if recursive:
                    folders.extend(Util.find_folders_by_extension(file_path, recursive))
        return folders

    @staticmethod
    def run_and_wait_by_pid(command):
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        process.wait()
        output_stdout = process.stdout
        output_stderr = process.stderr
        return output_stdout, output_stderr

    @staticmethod
    def write_json_file(file_path, data):
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)

    @staticmethod
    def read_json_file(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)

    @staticmethod
    def ensure_dir_exists(dir_path):
        os.makedirs(dir_path, exist_ok=True)

## 3. AudioCompiler Class

In [3]:
class AudioCompiler:

    def __init__(self):
        self.max_duration = 60 * 12  # 12 Minutes
        self.target_dbfs = [-0.1]
        self.src_dir = None
        self.dst_dir = None
        self.aug_dir = None
        self.audio_delim_dir = None
        self.spoken_digits_dir = None
        self.dry_run = True  # Default to dry run for safety

    def set_max_duration(self, max_duration):
        self.max_duration = max_duration
        return self

    def set_src_dir(self, src_dir):
        self.src_dir = src_dir
        if not self.dry_run:
            Util.ensure_dir_exists(src_dir)
        return self

    def set_dst_dir(self, dst_dir):
        self.dst_dir = dst_dir
        if not self.dry_run:
            Util.ensure_dir_exists(dst_dir)
        return self

    def set_aug_dir(self, aug_dir):
        self.aug_dir = aug_dir
        if not self.dry_run:
            Util.ensure_dir_exists(aug_dir)
        return self

    def set_target_dbfs(self, target_dbfs):
        self.target_dbfs = target_dbfs
        return self

    def set_audio_delim_dir(self, audio_delim_dir):
        self.audio_delim_dir = audio_delim_dir
        return self

    def set_spoken_digits_dir(self, spoken_digits_dir):
        self.spoken_digits_dir = spoken_digits_dir
        return self

    def set_dry_run(self, dry_run):
        self.dry_run = dry_run
        return self

    def combine_audio_files(self, audio_files, output_file):
        audio_segments = AudioSegment.empty()
        for file in audio_files:
            audio_segment = AudioSegment.from_wav(file)
            audio_segments += audio_segment
        audio_segments.export(output_file, format='wav')

    def adjust_max_dBFS(self, audio_sample, target_max_dBFS):
        return audio_sample.apply_gain(-audio_sample.max_dBFS + target_max_dBFS)

    def augment_samples(self, files):
        ret = []
        for file in files:
            if not os.path.exists(file):
                print(f"Source {file} does not exist.")
                break
            audio_sample = None
            for target_max_dBFS in self.target_dbfs:
                target_file = file.replace(self.src_dir, self.aug_dir)
                max_dBFS_str = "maxdBFS-{:0>4.1f}".format(-target_max_dBFS)
                target_file = target_file.replace(".wav", f"_{max_dBFS_str}.wav")
                ret.append(target_file)
                if os.path.exists(target_file):
                    print(f"Target {target_file} already exists.")
                    continue
                if self.dry_run:
                    print(f"[DRY RUN] Would process {file} -> {target_file}")
                    continue
                if audio_sample == None:
                    print(f"Processing {file}")
                    os.makedirs(os.path.dirname(target_file), exist_ok=True)
                    y, sr = librosa.load(file)
                    audio_sample = AudioSegment.from_file(file)
                audio_sample = self.adjust_max_dBFS(audio_sample, target_max_dBFS)
                audio_sample.export(target_file, format="wav")
        print("Done.")
        return ret

## 4. Main Processing Pipeline

In [4]:
def run_audio_compiler(compiler: AudioCompiler):
    """Main processing pipeline for the AudioCompiler."""
    
    dry_run = compiler.dry_run
    prefix = "[DRY RUN] " if dry_run else ""
    
    if dry_run:
        print("=" * 60)
        print("DRY RUN MODE - No files will be created or modified")
        print("=" * 60)
        print("")
    
    if not dry_run:
        Util.ensure_dir_exists(compiler.aug_dir)
    
    # Check if source directory exists
    if not os.path.exists(compiler.src_dir):
        print(f"{prefix}Source directory does not exist: {compiler.src_dir}")
        print(f"{prefix}Please create the directory and add audio samples.")
        return
    
    files = Util.find_files_by_extension(compiler.src_dir, 'wav', recursive=True)
    files.sort()
    print(f"{prefix}Found {len(files)} wav files in {compiler.src_dir}")
    
    if len(files) == 0:
        print(f"{prefix}No audio files found. Nothing to do.")
        return

    print(f"{prefix}Checking all {len(files)} files for duration...")
    tmp = []
    total_duration = 0
    for file in files:
        audio_segment = AudioSegment.from_wav(file)
        duration = audio_segment.duration_seconds
        if duration > compiler.max_duration:
            print(f"{prefix}File {file} is too long: {duration} seconds")
            continue
        total_duration += duration
        tmp.append(file)
    files = tmp

    print("")
    print(f"{prefix}Found {len(files)} files with duration less than {compiler.max_duration} seconds")
    print(f"{prefix}Total duration: {total_duration:.2f} seconds or {total_duration / 60:.2f} minutes")

    print(f"{prefix}Augmenting samples...")
    files = compiler.augment_samples(files)
    files.sort()

    # In dry run mode, the augmented files don't exist, so we use the original files for stats
    if dry_run:
        print(f"{prefix}Would create {len(files)} augmented files")
        print("")
        print(f"{prefix}Summary:")
        print(f"{prefix}  - Input files: {len(tmp)}")
        print(f"{prefix}  - Augmented files: {len(files)}")
        print(f"{prefix}  - Total duration: {total_duration:.2f} seconds ({total_duration / 60:.2f} minutes)")
        estimated_batches = int(total_duration * len(compiler.target_dbfs) / compiler.max_duration) + 1
        print(f"{prefix}  - Estimated output batches: ~{estimated_batches}")
        print(f"{prefix}  - Output directory: {compiler.dst_dir}")
        print("")
        print("To run for real, set: ac.set_dry_run(False)")
        return

    file_metadata = {}
    total_duration = 0
    for file in files:
        audio_segment = AudioSegment.from_wav(file)
        duration = audio_segment.duration_seconds
        if duration > compiler.max_duration:
            print("File", file, "is too long:", duration, "seconds")
            continue
        total_duration += duration
        file_metadata[file] = {
            'duration': duration
        }

    print("")
    print("Found", len(files), "files with duration less than", compiler.max_duration, "seconds")
    print("Total duration:", "{:.2f}".format(total_duration), "seconds or",
          "{:.2f}".format(total_duration / 60), "minutes")

    # Helper functions for file naming and metadata
    def getCombinedTargetFile(batch_number):
        return compiler.dst_dir + '/combined_files_' + str(batch_number).zfill(3) + '.wav'

    def getCombinedTargetMetadataFile(batch_number):
        return compiler.dst_dir + '/combined_files_' + str(batch_number).zfill(3) + '.json'

    def writeMetadataFile(metadata, batch_number):
        outFile = getCombinedTargetMetadataFile(batch_number)
        Util.write_json_file(outFile, metadata)

    def newMetadata():
        return {
            'segments': [],
        }

    def newFileMetadata(type, file, start_seconds, duration_seconds):
        return {
            'type': type,  # 'AUDIO' or 'DELIMITER'
            'file': file,
            'start': start_seconds,
            'duration': duration_seconds,
            'end': start_seconds + duration_seconds
        }

    def append_audio_spoken_digits(audio_segments, audio_metadata, number: str, voice: str = "lucas",
                                   variant: int = 0):
        number = str(number)
        for digit in number:
            if digit not in '1234567890':
                raise ValueError(f"Invalid digit: {digit}")

        for digit in number:
            file = compiler.spoken_digits_dir + f"/{digit}_{voice}_{variant}.wav"
            if not os.path.exists(file):
                raise ValueError(f"File not found: {file}")
            if audio_segments is None:
                audio_segments = AudioSegment.empty()
                audio_metadata = newMetadata()
            audio_segment = AudioSegment.from_wav(file)
            audio_metadata['segments'].append(
                newFileMetadata(
                    "SPOKEN-DIGIT",
                    file,
                    start_seconds=audio_segments.duration_seconds,
                    duration_seconds=audio_segment.duration_seconds,
                )
            )
            audio_segments += audio_segment
        return audio_segments, audio_metadata

    def load_audio_delimiter(name):
        file = compiler.audio_delim_dir + '/' + name + '.wav'
        if not os.path.exists(file):
            raise Exception("Delimiter file not found: " + file)
        audio_segment = AudioSegment.from_wav(file)
        return file, audio_segment

    def append_audio_start_marker(name, audio_segments, audio_metadata):
        delimiter_sequence = [
            'silence-0.5s',
            name,
            'silence-0.5s',
            name,
            'silence-0.5s',
            name,
            'silence-0.5s'
        ]
        for delimiter in delimiter_sequence:
            delimiter_file, audio_segment = load_audio_delimiter(delimiter)
            if audio_segments is None:
                audio_segments = AudioSegment.empty()
                audio_metadata = newMetadata()
            audio_metadata['segments'].append(
                newFileMetadata(
                    "DELIMITER",
                    delimiter_file,
                    start_seconds=audio_segments.duration_seconds,
                    duration_seconds=audio_segment.duration_seconds,
                )
            )
            audio_segments += audio_segment
        return audio_segments, audio_metadata

    def append_audio_delimiter(delimiter_name, audio_segments, audio_metadata):
        if audio_segments is not None:
            delimiter_file, audio_segment = load_audio_delimiter(delimiter_name)
            audio_metadata['segments'].append(
                newFileMetadata(
                    "DELIMITER",
                    delimiter_file,
                    start_seconds=audio_segments.duration_seconds,
                    duration_seconds=audio_segment.duration_seconds,
                )
            )
            audio_segments += audio_segment

        return audio_segments, audio_metadata

    def append_audio_segment(fileName, audio_segments, audio_metadata, batch_number):
        if not fileName:  # just write to disk
            if audio_segments is not None:
                audio_metadata['duration'] = audio_segments.duration_seconds
                audio_metadata['total_segments'] = len(audio_metadata['segments'])
                writeMetadataFile(audio_metadata, batch_number)
                target_file = getCombinedTargetFile(batch_number)
                audio_segments.export(target_file, format='wav')
                print("Created", target_file)
                batch_number += 1
            return None, None, batch_number

        if os.path.exists(fileName) is False:
            raise ValueError("File not found: " + fileName)

        audio_segment = AudioSegment.from_wav(fileName)
        if audio_segments is not None:
            total_duration = audio_segments.duration_seconds + audio_segment.duration_seconds
            if total_duration > compiler.max_duration:
                audio_metadata['duration'] = audio_segments.duration_seconds
                audio_metadata['total_segments'] = len(audio_metadata['segments'])
                writeMetadataFile(audio_metadata, batch_number)

                target_file = getCombinedTargetFile(batch_number)
                audio_segments.export(target_file, format='wav')
                print("Created", target_file)
                batch_number += 1
                audio_segments = audio_metadata = None

        if audio_segments is None:
            audio_segments = AudioSegment.empty()
            audio_metadata = newMetadata()

            number = str(batch_number).zfill(3)  # spoken number
            audio_segments, audio_metadata = append_audio_spoken_digits(audio_segments, audio_metadata, number)
            audio_segments, audio_metadata = append_audio_start_marker('beep-07a', audio_segments, audio_metadata)

        audio_metadata['segments'].append(
            newFileMetadata(
                'AUDIO',
                fileName,
                start_seconds=audio_segments.duration_seconds,
                duration_seconds=audio_segment.duration_seconds,
            )
        )
        audio_segments += audio_segment

        # Add delimiter at the end
        audio_segments, audio_metadata = append_audio_delimiter('silence-0.5s', audio_segments, audio_metadata)
        audio_segments, audio_metadata = append_audio_delimiter('beep-07a', audio_segments, audio_metadata)
        audio_segments, audio_metadata = append_audio_delimiter('silence-0.5s', audio_segments, audio_metadata)
        return audio_segments, audio_metadata, batch_number

    print("Combining files...")
    audio_segments, audio_metadata, batch_number = None, None, 0
    for fileCounter, file in enumerate(file_metadata):
        audio_segments, audio_metadata, batch_number = append_audio_segment(file, audio_segments, audio_metadata,
                                                                            batch_number)
        if fileCounter % 25 == 0 and fileCounter > 0:
            print("Processed file", fileCounter, "of", len(file_metadata),
                        "{:.2f}".format(100 * fileCounter / len(file_metadata)), "%")
        # delete file if in aug_dir
        if compiler.aug_dir in file:
            print("removing", file)
            os.remove(file)
    append_audio_segment(None, audio_segments, audio_metadata, batch_number)  # write last content to disk
    
    # Clean up augmentation directory
    if os.path.exists(compiler.aug_dir) and not os.listdir(compiler.aug_dir):
        os.rmdir(compiler.aug_dir)
    
    print("Done")

## 5. Configuration

Load configuration from the project's Config class.

In [5]:
# Load configuration from config.json
cfg = Config()  # Auto-locates config.json in project root
cfg.print_paths()

Bandsalat Configuration:
  Project root:      /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat
  Data:              /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data
  Models:            /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/models
  Audio:             /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio
  Audio assets:      /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/assets
  Datasets:          /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/datasets
  Datasets workdir:  /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/datasets-workdir


## 6. Run the Compiler

In [6]:
# Initialize and configure the compiler using Config paths
ac = AudioCompiler()

# Set paths from Config
# Note: audio_assets contains both delimiters and spoken digits subdirectories
audio_assets = cfg.get_audio_assets_dir()

ac.set_src_dir(str(audio_assets / "samples"))  # Source samples to compile
ac.set_dst_dir(str(cfg.get_datasets_workdir() / "compilation"))  # Output combined files
ac.set_aug_dir(str(cfg.get_datasets_workdir() / "tmp"))  # Temporary augmentation dir
ac.set_audio_delim_dir(str(audio_assets / "audio_delimiters"))
ac.set_spoken_digits_dir(str(audio_assets / "free-spoken-digit-dataset"))
ac.set_max_duration(720)  # 12 minutes
ac.set_target_dbfs([-0.1, -3, -6, -12, -24, -48])

# DRY RUN is enabled by default - no files will be created
# To actually run, uncomment the next line:
ac.set_dry_run(False)

print("AudioCompiler Configuration:")
print(f"  Dry run:             {ac.dry_run}")
print(f"  Source directory:    {ac.src_dir}")
print(f"  Output directory:    {ac.dst_dir}")
print(f"  Augmentation dir:    {ac.aug_dir}")
print(f"  Audio delimiters:    {ac.audio_delim_dir}")
print(f"  Spoken digits:       {ac.spoken_digits_dir}")
print(f"  Max duration:        {ac.max_duration} seconds ({ac.max_duration/60:.1f} minutes)")
print(f"  Target dBFS levels:  {ac.target_dbfs}")

AudioCompiler Configuration:
  Dry run:             True
  Source directory:    /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/assets/samples
  Output directory:    /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/datasets-workdir/compilation
  Augmentation dir:    /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/datasets-workdir/tmp
  Audio delimiters:    /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/assets/audio_delimiters
  Spoken digits:       /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/assets/free-spoken-digit-dataset
  Max duration:        720 seconds (12.0 minutes)
  Target dBFS levels:  [-0.1, -3, -6, -12, -24, -48]


In [7]:
# Run the audio compilation pipeline
# By default, this runs in DRY RUN mode - no files will be created or modified
# To run for real, set ac.set_dry_run(False) in the cell above
run_audio_compiler(ac)

DRY RUN MODE - No files will be created or modified

[DRY RUN] Found 12 wav files in /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/assets/samples
[DRY RUN] Checking all 12 files for duration...

[DRY RUN] Found 12 files with duration less than 720 seconds
[DRY RUN] Total duration: 3.62 seconds or 0.06 minutes
[DRY RUN] Augmenting samples...
[DRY RUN] Would process /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/assets/samples/Acoustic Hat-01.wav -> /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/datasets-workdir/tmp/Acoustic Hat-01_maxdBFS-00.1.wav
[DRY RUN] Would process /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/assets/samples/Acoustic Hat-01.wav -> /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/datasets-workdir/tmp/Acoustic Hat-01_maxdBFS-03.0.wav
[DRY RUN] Would process /Users/flo/Developer/github/xmas-hackathon-2025-bandsalat/data/audio/assets/samples/Acoustic Hat-01.wa